# Synthetic model

### Brief description

This Jupyter notebook is a supplementary material contaning the code needed to generate part of the synthetic data and reproduce part of the figures and results of the manuscript. This code uses the open-source Python toolkit for geophysical modeling and inversion [Fatiando a Terra](http://www.fatiando.org/index.html).

This code creates the simulated bodies used in the numerical applications.

**Note:** A Jupyter notebook is a document that incorporates text, results, figures, and the Python source code to produce them. If you are viewing this from the URL https://nbviewer.jupyter.org/, the document will be static (not executable). To run the code, you will need to download the notebook and install all the required dependencies. For installing the Fatiando a Terra and all the required dependencies, please follow the instructions shown in http://www.fatiando.org/install.html.

### Import the required dependencies

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import datetime
from IPython.display import Image as img
from IPython.display import Markdown as md
from IPython.display import display as dp
import string as st

from __future__ import division

#import auxiliary_functions as af

import fatiando
from fatiando import gridder
from fatiando.mesher import Prism
from fatiando.gridder import regular
from fatiando.gravmag import prism
from fatiando.vis import myv
from fatiando.utils import ang2vec, contaminate
from fatiando.constants import G

plt.rc('font', size=14)

C:\Anaconda2\lib\site-packages\fatiando\vis\mpl.py:70: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
C:\Anaconda2\lib\site-packages\fatiando\vis\myv.py:53: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


In [2]:
notebook_name = 'synthetic_model_Single_Prims.ipynb'

In [3]:
print("Version of Fatiando a Terra used: {}".format(fatiando.__version__))

Version of Fatiando a Terra used: ac2afbcb2d99b18f145cc1ed40075beb5f92dd5a


In [4]:
saved_files = [] #list of saved files

### Synthetic model formed by two rectangular prisms

#### Create the model

In [5]:
rho1 = 2670.   # density of the smaller prism
m1 = 10.       # magnetization intensity of the smaller prism
incs = 30.     # magnetization inclination of the source
decs = -10.0   # magnetization declination of the source

# Cartesian components of the unitary vector with the
# same direction as the magnetization of the sources
mx, my, mz = ang2vec(1.0, incs, decs)

In [6]:
full_model_single = dict() #empty dictionary
full_model_single_pole   = dict() #empty dictionary

In [7]:
#Local-geomagnetic field (inclination and declination in degress)
#full_model_single['geomag_field'] = (6., -40.5)

In [8]:
#Create a model composed of two elements of the class fatiado.mesher.Prism
#full_model_single['model'] = [Prism(2000., 3000., -2000., 2000., 500., 1000.,
#                       props={'density':rho1, 'magnetization': ang2vec(m1,incs,decs)})]

full_model_single['model'] = [Prism(2200., 2800., -1000., 1000., 500., 1000.,
                       props={'density':rho1, 'magnetization': ang2vec(m1,incs,decs)})]

In [9]:
#Create a model in the pole composed of two elements of the class fatiado.mesher.Prism

#full_model_single_pole['model_pole'] = [Prism(2000., 3000., -2000., 2000., 500., 1000.,
#                       props={'density':rho1, 'magnetization': ang2vec(m1,90.,0.)})]

full_model_single_pole['model_pole'] = [Prism(2200., 2800., -1000., 1000., 500., 1000.,
                       props={'density':rho1, 'magnetization': ang2vec(m1,90.,0.)})]

In [10]:
#horizontal projection of the model
full_model_single['projection'] = []
for i, p in enumerate(full_model_single['model']):
    full_model_single['projection'].append(p.get_bounds()[:4])

In [11]:
#horizontal projection of the model Pole
full_model_single_pole['projection_pole'] = []
for i, p in enumerate(full_model_single_pole['model_pole']):
    full_model_single_pole['projection_pole'].append(p.get_bounds()[:4])

In [12]:
full_model_single['projection']

[[2200.0, 2800.0, -1000.0, 1000.0]]

In [13]:
assert (full_model_single['projection'] == full_model_single_pole['projection_pole']), 'The synthetic model must be equal of the synthetic model in the pole '

In [14]:
#top and bottoms of the model
full_model_single['depth_model'] = []
for i, depth in enumerate(full_model_single['model']):
    full_model_single['depth_model'].append(depth.get_bounds()[4:])
print  full_model_single['depth_model']

[[500.0, 1000.0]]


In [15]:
#auxiliary informations about the model
now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
full_model_single['metadata'] = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)

In [16]:
#auxiliary informations about the model in the pole
now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
full_model_single_pole['metadata'] = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)

#### Save the model

In [17]:
#save the model by using the Python pickle module (https://docs.python.org/2/library/pickle.html)
file_name = '..\data\model_single.pickle'
with open(file_name, 'w') as f:
    pickle.dump(full_model_single, f)
    
saved_files.append(file_name)

In [18]:
#save the model by using the Python pickle module (https://docs.python.org/2/library/pickle.html)
file_name = '..\data\model_single_pole.pickle'
with open(file_name, 'w') as f:
    pickle.dump(full_model_single_pole, f)
    
saved_files.append(file_name)

#### 3D plot of the model

bounds = [11000, 20000, 13000, 24000, 0, 3000]
scene = myv.figure(size=(12000., 8000.))
myv.prisms(full_model_single['model'], color=(0.5, 0.5, 0.5), linewidth=1)
ax = myv.axes(myv.outline(extent=bounds), ranges=[b*0.001 for b in bounds], nlabels=3, fmt='%.1f')
ax.axes.x_label, ax.axes.y_label, ax.axes.z_label = 'x (km)', 'y (km)', 'z (km)'
ax.axes.font_factor = 1.1
myv.wall_north(bounds)
myv.wall_bottom(bounds)

scene.scene.camera.position = [-3636.7651215354613, 10743.121645670348, -8829.4824339882798]
scene.scene.camera.focal_point = [15499.999999999745, 18499.999999999778, 1499.9999999999959]
scene.scene.camera.view_angle = 30.0
scene.scene.camera.view_up = [0.41366230359938, 0.17041624454596982, -0.89433875135526963]
scene.scene.camera.clipping_range = [10422.897651780093, 39087.786072184674]
scene.scene.camera.compute_view_plane_normal()
scene.scene.render()

file_name = '..\\manuscript\\figs\\Fig3'

myv.savefig(file_name+'.png')

saved_files.append(file_name+'.png')

myv.show()

caption = "Synthetic prism with constant density $\\rho$ = %.1f kg/m$^{3}$ and constant magnetization vector \
with intensity $m$ = %.1f A/m, \
inclination %.1f$^{\circ}$ and declination %.1f$^{\circ}$. The geomagnetic field has %.1f$^{\circ}$ \
inclination and %.1f$^{\circ}$ \
declination." % (rho1, m1, incs, decs, full_model_single['geomag_field'][0], full_model_single['geomag_field'][1])

dp(md(caption))

### Saved files

In [19]:
with open('reports\\report_%s.md' % notebook_name[:st.index(notebook_name, '.')], 'w') as f:
    f.write('# Saved files \n')
    now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
    header = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)
    f.write('\n\n'+header+'\n\n')
    for i, sf in enumerate(saved_files):
        print '%d  %s' % (i+1,sf)
        f.write('*  `%s` \n' % (sf))

1  ..\data\model_single.pickle
2  ..\data\model_single_pole.pickle
